# 🥋 Lekcja 5: In-Place Operations (Oszczędność vs Ryzyko)

W PyTorch operacje zakończone podkreślnikiem `_` (np. `add_`, `scatter_`, `relu_`) lub operatory przypisania (`+=`, `*=`) działają **In-Place**.

**Zaleta:** Nie alokują nowej pamięci. Działają na istniejącym buforze.
**Wada:** Nadpisują dane, które mogą być potrzebne do obliczenia gradientu.

Jeśli nadpiszesz tensor, który był potrzebny do `backward()`, PyTorch wykryje to i rzuci słynnym błędem:
`RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation`.

W tej lekcji nauczymy się, kiedy można, a kiedy nie wolno tego robić.

In [1]:
import torch

# Funkcja do sprawdzania adresu pamięci
def check_memory(name, old_ptr, tensor):
    new_ptr = tensor.untyped_storage().data_ptr()
    if old_ptr == new_ptr:
        print(f"✅ {name}: Adres BEZ ZMIAN (In-Place). Oszczędzamy pamięć.")
    else:
        print(f"❌ {name}: Nowy adres (Out-of-Place). Alokacja pamięci.")
    return new_ptr

# Baza
t = torch.ones(1000, 1000)
ptr = t.untyped_storage().data_ptr()
print(f"Startowy adres: {ptr}")

Startowy adres: 5292439961600


## Test Pamięci: `x = x + 1` vs `x += 1`

Sprawdźmy, co dzieje się w pamięci RAM.

In [2]:
# 1. Out-of-place (Standard)
t = t + 1
ptr = check_memory("x = x + 1", ptr, t)

# 2. In-place (Pythonowy operator)
t += 1
ptr = check_memory("x += 1", ptr, t)

# 3. In-place (Metoda PyTorch z _)
t.add_(1)
ptr = check_memory("x.add_(1)", ptr, t)

# 4. Out-of-place (Metoda PyTorch bez _)
t = t.add(1)
ptr = check_memory("x.add(1)", ptr, t)

❌ x = x + 1: Nowy adres (Out-of-Place). Alokacja pamięci.
✅ x += 1: Adres BEZ ZMIAN (In-Place). Oszczędzamy pamięć.
✅ x.add_(1): Adres BEZ ZMIAN (In-Place). Oszczędzamy pamięć.
❌ x.add(1): Nowy adres (Out-of-Place). Alokacja pamięci.


## The Dark Side: Autograd i Version Counter

Każdy tensor w PyTorch ma licznik wersji (`_version`).
*   Przy każdej operacji In-Place licznik rośnie.
*   Autograd zapisuje sobie: "Potrzebuję tensora X w wersji 0, żeby policzyć pochodną".
*   Jeśli przy `backward()` okaże się, że tensor X ma teraz wersję 1 (bo go nadpisałeś), PyTorch rzuca błędem, zamiast liczyć głupoty.

In [5]:
# Symulacja błędu w treningu

# Wagi (wymagają gradientu) - to jest "Leaf Variable"
w = torch.tensor([5.0], requires_grad=True)

# Krok 1: Forward
# y = w * 2
y = w * 2

print(f"Wartość y przed zmianą: {y}")

# Krok 2: Operacja In-Place na 'w' (PSUCIE DANYCH!)
# Używamy no_grad(), żeby zmusić PyTorch do wykonania operacji na Liściu.
# To symuluje np. aktualizację wag przez optymalizator w złym momencie.
with torch.no_grad():
    w *= 100 

print(f"Wartość w (zmieniona): {w}")

# Krok 3: Backward
# Teraz PyTorch spróbuje policzyć pochodną.
# Powinien zauważyć, że 'w' (które było potrzebne do obliczeń) zmieniło się pod jego nosem.
try:
    y.backward()
except RuntimeError as e:
    print("\n🚫 ZŁAPANO BŁĄD AUTOGRADU:")
    print(e)

Wartość y przed zmianą: tensor([10.], grad_fn=<MulBackward0>)
Wartość w (zmieniona): tensor([500.], requires_grad=True)


## Bezpieczne In-Place (ReLU)

Są operacje, które **można** robić In-Place.
Klasycznym przykładem jest `ReLU`.
$$ f(x) = \max(0, x) $$

Pochodna ReLU zależy od tego, czy $x > 0$.
Możemy nadpisać $x$ wynikiem, bo informacja o znaku (czy było > 0) jest zachowana w wyniku (jeśli wynik > 0, to wejście też było > 0).
Dlatego `nn.ReLU(inplace=True)` jest bezpieczne i zalecane.

In [8]:
# 1. Dane wejściowe (Leaf)
x = torch.tensor([-5.0, 2.0], requires_grad=True)

# 2. Symulacja warstwy (Operacja pośrednia)
# Klonujemy x. Teraz 'h' to nie jest liść, to jest "wynik operacji clone".
# Na wynikach operacji MOŻNA robić in-place!
h = x.clone()

# 3. Bezpieczne In-Place (ReLU) na zmiennej pośredniej
# Modyfikujemy 'h' bezpośrednio w pamięci
torch.relu_(h) 

print(f"Wynik po ReLU: {h}")

# 4. Backward
try:
    h.sum().backward()
    print("✅ ReLU in-place przeszło backward!")
except Exception as e:
    print(f"Błąd: {e}")

# Sprawdźmy gradient na oryginale
# Dla -5.0 gradient powinien być 0.
# Dla 2.0 gradient powinien być 1.
print(f"Gradient x: {x.grad}")

Wynik po ReLU: tensor([0., 2.], grad_fn=<ReluBackward0>)
✅ ReLU in-place przeszło backward!
Gradient x: tensor([0., 1.])


## 🥋 Black Belt Summary

1.  **Dla optymalizacji:** Używaj `+=`, `*=`, `add_()`, `scatter_()` tam, gdzie **nie potrzebujesz gradientów** (np. przy aktualizacji wag w optymalizatorze `w -= lr * grad`).
2.  **Dla bezpieczeństwa:** Unikaj In-Place na tensorach, które są częścią grafu obliczeniowego (między wejściem a Loss), chyba że wiesz, co robisz (np. ReLU).
3.  **Debugowanie:** Jeśli widzisz błąd `modified by an inplace operation`, zamień `x += y` na `x = x + y`. To zazwyczaj naprawia problem (kosztem pamięci).